# LangChain Basics

## Install dependencies

In [ ]:
pip install -r ./requirements.txt

### Verify if LangChain is installed

In [ ]:
pip show langchain

#### Python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (Cohere, Pinecone) from .env 
load_dotenv(find_dotenv(), override=True)

COHERE_API_KEY = os.environ.get('COHERE_API_KEY')
print(COHERE_API_KEY)

## Early Experimentation with LLMs Using LangChain 

### Introduction

LangChain is a framework for developing applications powered by language models. It enables applications that:

- Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)

- Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

### LLM Models (Wrappers): Cohere

https://python.langchain.com/docs/integrations/llms/cohere

Cohere is a Canadian startup that provides natural language processing models that help companies improve human-machine interactions.

- Founded in 2020
- Based in Toronto, Canada 

#### Setting up the LLM

In [ ]:
from langchain.llms import Cohere

# Create Cohere LLM instance with parameters
llm = Cohere(
            # A non-negative float that tunes the degree of randomness in generation.
            # Higher values give more diverse, creative responses
            # Lower values give more focused, deterministic responses
            temperature=0.7, 
            # The maximum number of tokens to generate in the response
            max_tokens=512, 
            cohere_api_key=COHERE_API_KEY)
print(llm)

#### First try - test Cohere

In [ ]:
# Generate response from LLM by invoking it 
output = llm.invoke('explain 5G networks in one sentence')
print(output)

In [ ]:
# Pass prompt to LLM's get_num_tokens method
num_tokens = llm.get_num_tokens('explain 5G networks in one sentence')

# Print number of tokens in the tokenized prompt.
print(num_tokens)

In [ ]:
# Prompt LLM with two questions 
prompts = ['What is the date of the European Union creation.', 
           'What is the formula for the area of a room?']

# Generate responses to prompts
# In this example, we can't use invoke() -> we don't have a single prompt
output = llm.generate(prompts)  

# Print list of generated responses
print(output.generations)

#### Example in order to extract the answer of the  first question

In [ ]:
print(output.generations[0][0].text)

#### understand the difference between invoke() and generate() - check here

https://chat.langchain.com/  
https://python.langchain.com/docs/get_started/introduction
https://api.python.langchain.com/en/latest/langchain_api_reference.html

## Deep dive into LangChain 

### Prompt Templates

https://python.langchain.com/docs/modules/model_io/prompts/quick_start

In [ ]:
from langchain import PromptTemplate

# Define template with input variables in braces
template = '''You are an experienced mathematician professor.
Write a few sentences about the following mathematical {concept} in {language}.'''

# Create PromptTemplate object with input variables
prompt = PromptTemplate(
    input_variables=['concept', 'language'], # list of input variable names
    template=template # template string
)

print(prompt)

In [ ]:
from langchain.llms import Cohere

# Create Cohere LLM instance with parameters
llm = Cohere(temperature=0.7, max_tokens=512, cohere_api_key=COHERE_API_KEY)
print(llm)

# Format prompt template with input values 
output = llm(prompt.format(concept='Pythagorean theorem', language='Spanish'))
print(output)

### Learn about Chains

Resources :

https://python.langchain.com/docs/modules/chains  
https://chat.langchain.com/

#### Simple Chains

In [ ]:
from langchain.chat_models import ChatCohere
from langchain import PromptTemplate
from langchain.chains import LLMChain

# Create ChatCohere LLM instance
llm = ChatCohere(temperature=0.5, cohere_api_key=COHERE_API_KEY)

# Define template string 
template = '''You are an experienced mathematician professor.
Write a few sentences about the following mathematical {concept} in {language}.'''

# Create PromptTemplate 
prompt = PromptTemplate(
    input_variables=['concept', 'language'],
    template=template
)

# Create LLMChain with llm and prompt
chain = LLMChain(llm=llm, prompt=prompt)

# Run chain with input values
output = chain.invoke({'concept': 'Pythagorean theorem', 'language': 'Spanish'})
print(output)

#### Sequential Chains

In [ ]:
from langchain.chat_models import ChatCohere
from langchain.llms import Cohere
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Create Cohere LLM instance with parameters
llm1 = Cohere(temperature=0.2, max_tokens=512, cohere_api_key=COHERE_API_KEY)

# Create first prompt template
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)

# Create first LLMChain with llm1 and prompt1
chain1 = LLMChain(llm=llm1, prompt=prompt1)


# Create ChatCohere LLM instance
llm2 = ChatCohere(temperature=0.9, cohere_api_key=COHERE_API_KEY)

# Create second prompt template
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)

# Create second LLMChain with llm2 and prompt2
chain2 = LLMChain(llm=llm2, prompt=prompt2)

# Create SequentialChain using the two chains 
# You can check what verbose means with :
# https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html?highlight=simplesequentialchain%20verbose
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Run the chain with the input 'softmax'
output = overall_chain.run('softmax')



#### LangChain Agents

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import Cohere

# Create Cohere LLM instance
llm = Cohere(temperature=0, cohere_api_key=COHERE_API_KEY)

# Create agent by passing LLM, tool (what is it !?)
# You can check what verbose means with :
# https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html?highlight=simplesequentialchain%20verbose
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Run agent with input command 
agent_executor.run('what is the answer to x**3 + 3*x**2 - 5 with x = 2157625176 ?')

